### Before

#### Об этом ноутбуке
Данный ноутбук является финальной частью.
Он аккумулирует в себе предыдущие два + исследует решения коллег по цеху, для улучшения навыков в будущем.

#### Цели данного ноутбука:
- Провести "работу над ошибками" после участия [соревновании](https://www.kaggle.com/competitions/adengi-internship);
- Подробно разобрать различные методы, которые используют коллеги для анализа ио бработки данных (методы заимствованы из этих ноутбуков: [Ноутбук1](https://www.kaggle.com/code/korcy78/baseline), [Ноутбук2](https://www.kaggle.com/code/alexaali/catboost-0-91-score))

#### Description
В этом соревновании вам нужно будет натренировать модель оттока, которая лучше всех сможет генерализовать общую зависимость оттока.

#### Evaluation
Ответы модели будут оценены с помощью F1 метрики между предсказанными ответами и реальными.

#### Первые мысли
По сути стандартная задача бинарной классификаци.

Необходимо предсказать отток клиентов.

В зависимости от кол-ва данных, будем выбирать модель.

Предварительно, как многим известно, в подобных задачах себя очень хорошо зарекомендовали бустинги.
Я взглянул на файлик train.csv, он весит 2.2Гб, данных много, я думаю лучшим решением будут именно бустинги.

Но, т.к. я практикуюсь, мне не важно какой скор я выбью, лично мне важно попробовать разные подходы и поэкспериментировать.

Разумеется в рамках разумных сроков(решаю эту задачу только по выходным).

#### О данных
- monthly_income - среднемесячный заработок клиента (зарплата)
- payment_frequency - частота получения зарплаты (month - 1 раз в месяц, 2 weeks - раз в две недели, и тд)
- status - статус клиента (самозанятый, рабочий, и тд)
- work_experience - кол-во лет стажа клиента
- client_type - тип клиента (новый, повторный)
- settlement - город клиента
- requested_sum - запрашиваемая сумма клиента для займа, если interface - alfa
- region - регион клиента (область, округ, и тд)
- loan_id - уникальный идентификатор займа
- client_id - уникальный идентификатор клиента
- main_agreement_amount - основная одобренная сумма клиенту по займу (может быть больше, чем approved_amount)
- main_agreement_term - основной одобренный срок по займу
- requested_period_days - запрашиваемый срок по займу
- requested_amount - запрашиваемая сумма клиента по займу
- req_app_amount - разница между запрашиваемой суммой займа и одобренной
- approved_amount - одобренная сумма по займу
- source - канал привлечения клиента
- first_source - первый канал привлечения клиента
- period_days - период страховки по займу
- interface - интерфейс, откуда пришла заявка - (site, mobile)
- created_at - дата открытия займа
- type - тип займа (тип продукта)
- closed_at - дата закрытия займа
- days_finish_loan - время в днях, затраченное на закрытие займа (closed_at - created_at)
- gender - пол клиента
- ag - возраст клиента
- repayment_type - Тип комиссии по займу (с 2.5% - with_comission, 5% - with_big_comission, 0% - no_comission)
- loan_order - порядковый номер займа
- have_extension - имеется ли пролонгация по данному займу
- cnt_ext - кол-во пролонгаций по займу
- start_dt - дата начала (список) пролонгаций по займу
- term - срок пролонгации (список)
- price - цена пролонгации (список)
- elecs_sum - штрафы, пени
- recurents_sum - штрафы, пени (там вроде как различия в этапах их начисления)
- tamount - общий кэшфлоу клиента (общая сумма которая была на аккаунте клиента)
- issues - сумма просрочек, штрафов и пени
- principal - сумма основного долга
- interest - прибыль с клиента
- overdue_interest - прибыль с клиента, если есть прослойка
- overdue_fee - штрафы в просрочек
- contact_cases - кол-во обращений клиента с коллекшн
- nbki_score - скор клиента от рисков

- churn - колонка таргета

### Imports

In [132]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from category_encoders.cat_boost import CatBoostEncoder


import catboost as cb
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight


from sklearn.metrics import accuracy_score, classification_report, f1_score

import time
import warnings
from tqdm import tqdm
from datetime import datetime

from imblearn.over_sampling import SMOTE
warnings.filterwarnings("ignore")



# Utilities
from rapidfuzz import process, fuzz
from sklearn.base import BaseEstimator, TransformerMixin
import re
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

### Загрузка данных

In [91]:
start = time.time()
train = pd.read_csv('./data/train.csv')
elapsed = time.time() - start
print(f'Time is: {elapsed} seconds')

Time is: 21.179394960403442 seconds


In [92]:
start = time.time()
test = pd.read_csv('./data/test.csv')
elapsed = time.time() - start
print(f'Time is: {elapsed} seconds')

Time is: 6.376559019088745 seconds


#### Оптимизация данных

In [93]:
def optimize_data(df):
    # оптимизируем числовые данные
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')

    # Оптимизируем даты
    df["created_at"] = pd.to_datetime(df["created_at"])
    df["start_dt"] = pd.to_datetime(df["start_dt"])

    return df

In [94]:
# Размер трейна до оптимизации
print(f"Размер датасета до оптимизации: {train.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
train = optimize_data(train)
# Размер трейна после оптимизации
print(f"Размер датасета после оптимизации: {train.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

Размер датасета до оптимизации: 3231.96 MB
Размер датасета после оптимизации: 2150.51 MB


In [95]:
# Размер теста до оптимизации
print(f"Размер датасета до оптимизации: {test.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
test = optimize_data(test)
# Размер теста после оптимизации
print(f"Размер датасета после оптимизации: {test.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

Размер датасета до оптимизации: 971.89 MB
Размер датасета после оптимизации: 619.54 MB


### EDA

- База:
    - общая оценка данных
    - проверка на дубликаты;
    - обработка пропусков;
    - грамотная обработка категориальных переменных;
    - кодирование категориальных переменных;
    - знакомство с балансом целевой переменной;

### Общая оценка

In [96]:
train.shape

(4036207, 45)

In [97]:
test.shape

(1348743, 43)

Количество фичей в трейне и тесте различно больше чем на единицу, следовательно из теста удалили некий признак, который остался в трейне.

Оставлять его не имеет никакого смысла абсолютно)

Ознакомимся с данными детальнее

In [98]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4036207 entries, 0 to 4036206
Data columns (total 45 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   monthly_income         float32       
 1   work_experience        float32       
 2   requested_sum          float32       
 3   main_agreement_amount  float32       
 4   main_agreement_term    float32       
 5   requested_period_days  float32       
 6   requested_amount       float32       
 7   req_app_amount         float32       
 8   approved_amount        float32       
 9   period_days            float32       
 10  days_finish_loan       float32       
 11  ag                     float32       
 12  cnt_ext                float32       
 13  term                   float32       
 14  price                  float32       
 15  elecs_sum              float32       
 16  recurents_sum          float32       
 17  tamount                float32       
 18  issues                

In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348743 entries, 0 to 1348742
Data columns (total 43 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   monthly_income         1348680 non-null  float32       
 1   work_experience        463225 non-null   float32       
 2   requested_sum          163790 non-null   float32       
 3   main_agreement_amount  1348743 non-null  float32       
 4   main_agreement_term    1348743 non-null  float32       
 5   requested_period_days  1200882 non-null  float32       
 6   requested_amount       1209772 non-null  float32       
 7   req_app_amount         1209772 non-null  float32       
 8   approved_amount        1348743 non-null  float32       
 9   period_days            1348743 non-null  float32       
 10  days_finish_loan       1348743 non-null  float32       
 11  ag                     1348743 non-null  float32       
 12  cnt_ext                15170

In [99]:
#лишним является 'closed_at', без зазрений совести дропаем его из трейна
train.drop('closed_at', axis=1, inplace=True)

In [100]:
# так же есть задвоенный категориальный признак
test['client_type'] == test['client_type.1']

0          False
1          False
2          False
3          False
4          False
           ...  
1348738    False
1348739    False
1348740    False
1348741    False
1348742    False
Length: 1348743, dtype: bool

In [101]:
print(test['client_type'][:5], test['client_type.1'][:5], sep='\n')

0    1
1    1
2    0
3    1
4    1
Name: client_type, dtype: int8
0    repeated
1    repeated
2         new
3    repeated
4    repeated
Name: client_type.1, dtype: object


In [102]:
# избавимся и от него
train.drop('client_type.1', axis=1, inplace=True)
test.drop('client_type.1', axis=1, inplace=True)

In [103]:
train.shape[1] == test.shape[1] + 1

True

In [104]:
train.describe()

,monthly_income,work_experience,requested_sum,main_agreement_amount,main_agreement_term,requested_period_days,requested_amount,req_app_amount,approved_amount,period_days,...,interface,type,repayment_type,client_type,loan_order,have_extension,contact_cases,created_at,start_dt,churn
count,4.036000e+06,1.387847e+06,489817.000000,4.036207e+06,4.036207e+06,3.594589e+06,3.621371e+06,3.621371e+06,4.036207e+06,4.036207e+06,...,4.036207e+06,4.036207e+06,4.036207e+06,4.036207e+06,4.036207e+06,4.036207e+06,225917.000000,4036207,454277,4.036207e+06
mean,-8.252137e-05,1.327218e-04,-0.000607,-1.432685e-04,1.383396e-04,3.266359e-04,-2.585168e-04,-3.262011e-04,7.910570e-05,1.383396e-04,...,1.570352e+00,1.987750e+00,1.641676e+00,7.609392e-01,5.557003e+00,1.125505e-01,1.506301,2023-11-06 06:21:45.347923456,2023-11-17 00:42:24.480867072,2.938724e-01
min,-7.560081e-04,-2.522225e+00,-0.592014,-9.858535e-01,-1.145865e+00,-1.657011e+00,-1.318104e+00,-5.241477e+00,-1.152100e+00,-1.145865e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000,2022-04-19 06:50:05,2022-04-28 21:26:55,0.000000e+00
25%,-7.505928e-04,-7.846653e-01,-0.462179,-6.870721e-01,-3.406587e-01,-3.890285e-01,-8.330510e-01,-6.292958e-01,-7.512400e-01,-3.406587e-01,...,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,1.000000,2023-06-03 18:37:20,2023-06-25 21:29:29,0.000000e+00
50%,-7.488827e-04,8.411442e-02,-0.267427,-3.209154e-01,1.769741e-01,4.261034e-01,-8.342338e-02,-1.071620e-01,-2.167598e-01,1.769741e-01,...,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,1.000000,2023-11-22 08:52:46,2023-12-11 09:15:12,0.000000e+00
75%,-7.448212e-04,9.528942e-01,0.057159,5.080537e-01,1.769741e-01,4.261034e-01,1.195353e+00,6.760386e-01,5.181504e-01,1.769741e-01,...,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,7.000000e+00,0.000000e+00,1.000000,2024-04-27 15:37:10.500000,2024-04-25 05:45:35,1.000000e+00
max,1.339736e+03,9.528942e-01,5.899720,1.141358e+01,8.804188e+00,1.401163e+01,7.412853e+00,5.636309e+00,9.938363e+00,8.804188e+00,...,2.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,3.520000e+02,1.000000e+00,64.000000,2024-11-01 22:38:54,2024-11-02 07:49:56,1.000000e+00
std,9.431024e-01,9.999002e-01,0.997373,1.000089e+00,1.000341e+00,1.001271e+00,9.997053e-01,9.999505e-01,1.000235e+00,1.000341e+00,...,7.683009e-01,1.419784e-01,9.512670e-01,4.265100e-01,8.201449e+00,3.160425e-01,1.430518,NaN,NaN,4.555343e-01


In [105]:
train.head(5)

,monthly_income,work_experience,requested_sum,main_agreement_amount,main_agreement_term,requested_period_days,requested_amount,req_app_amount,approved_amount,period_days,...,client_type,settlement,region,gender,loan_order,have_extension,contact_cases,created_at,start_dt,churn
0,-0.000735,0.952894,NaN,-0.388291,-0.340659,-0.389028,-1.053530,-0.629296,-0.61762,-0.340659,...,1,г Москва,г Москва,male,2,0,NaN,2022-12-23 17:50:25,NaT,0
1,-0.000742,0.952894,NaN,-0.388291,-1.145865,-1.294731,-0.436189,-0.020140,-0.61762,-1.145865,...,1,г Москва,г Москва,female,14,0,NaN,2024-06-29 22:26:08,NaT,0
2,-0.000742,0.952894,NaN,-0.388291,-0.685747,-1.113590,-0.259806,0.153905,-0.61762,-0.685747,...,1,г Пермь,край Пермский,male,10,0,NaN,2023-04-10 21:06:38,NaT,0
3,-0.000742,0.952894,NaN,-0.388291,0.176974,-1.385301,1.239449,1.633284,-0.61762,0.176974,...,1,г Самара,обл Самарская,female,2,1,NaN,2023-10-01 00:13:16,2024-01-12 17:29:21,0
4,-0.000742,0.952894,NaN,-0.388291,-1.145865,-1.657011,0.357534,0.763061,-0.61762,-1.145865,...,1,г Москва,г Москва,female,17,0,NaN,2024-07-15 10:40:35,NaT,0


#### Дубли и пропуски.

In [106]:
print(f'Количество дубликатов: {train.duplicated().sum()}')

Количество дубликатов: 0


In [107]:
# пропуски в %
train.isnull().sum()/len(train)*100

monthly_income            0.005129
work_experience          65.615069
requested_sum            87.864423
main_agreement_amount     0.000000
main_agreement_term       0.000000
requested_period_days    10.941411
requested_amount         10.277867
req_app_amount           10.277867
approved_amount           0.000000
period_days               0.000000
days_finish_loan          0.000000
ag                        0.000000
cnt_ext                  88.760462
term                     88.760462
price                    88.766904
elecs_sum                 0.143551
recurents_sum             0.143551
tamount                   0.043952
issues                    0.000000
principal                 0.000000
interest                  0.000000
overdue_interest          0.000000
overdue_fee               0.000000
nbki_score                2.092955
payment_frequency         0.000000
status                    0.000000
loan_id                   0.000000
client_id                 0.000000
source              

Отличились следующие фичи:
- work_experience - стаж - 65% пропусков
- requested_sum - запрашиваемая сумма клиента для займа - 88% пропусков
- cnt_ext - кол-во пролонгаций по займу - 89% пропусков
- term - срок пролонгации (список) - 89% пропусков
- price -  цена пролонгации (список) - 89% пропусков
- contact_cases - кол-во обращений клиента с коллекшн - 94% пропусков
- start_dt  -  дата начала (список) пролонгаций по займу - 89% пропусков


Пропуски необходимо обработать, поэтому заполним их 0

In [108]:
# получим список столбцов с пропущенными значениями
nans = train.columns[train.isnull().any()].tolist()
print(nans)

['monthly_income', 'work_experience', 'requested_sum', 'requested_period_days', 'requested_amount', 'req_app_amount', 'cnt_ext', 'term', 'price', 'elecs_sum', 'recurents_sum', 'tamount', 'nbki_score', 'contact_cases', 'start_dt']


In [109]:
# 'start_dt' необходимо обработать отдельно
train[nans].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4036207 entries, 0 to 4036206
Data columns (total 15 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   monthly_income         float32       
 1   work_experience        float32       
 2   requested_sum          float32       
 3   requested_period_days  float32       
 4   requested_amount       float32       
 5   req_app_amount         float32       
 6   cnt_ext                float32       
 7   term                   float32       
 8   price                  float32       
 9   elecs_sum              float32       
 10  recurents_sum          float32       
 11  tamount                float32       
 12  nbki_score             float32       
 13  contact_cases          float32       
 14  start_dt               datetime64[ns]
dtypes: datetime64[ns](1), float32(14)
memory usage: 246.4 MB


In [110]:
nans.remove('start_dt')

In [111]:
train[nans] = train[nans].fillna(0)
test[nans] = test[nans].fillna(0)

In [112]:
# колонку с пропуском даты обработаем сегодняшней датой!
train['start_dt'].fillna(datetime.now(), inplace=True)
test['start_dt'].fillna(datetime.now(), inplace=True)

In [113]:
# пропуски в %
train.isnull().sum()/len(train)*100

monthly_income           0.0
work_experience          0.0
requested_sum            0.0
main_agreement_amount    0.0
main_agreement_term      0.0
requested_period_days    0.0
requested_amount         0.0
req_app_amount           0.0
approved_amount          0.0
period_days              0.0
days_finish_loan         0.0
ag                       0.0
cnt_ext                  0.0
term                     0.0
price                    0.0
elecs_sum                0.0
recurents_sum            0.0
tamount                  0.0
issues                   0.0
principal                0.0
interest                 0.0
overdue_interest         0.0
overdue_fee              0.0
nbki_score               0.0
payment_frequency        0.0
status                   0.0
loan_id                  0.0
client_id                0.0
source                   0.0
first_source             0.0
interface                0.0
type                     0.0
repayment_type           0.0
client_type              0.0
settlement    

In [115]:
# пропуски в %
test.isnull().sum()/len(train)*100

monthly_income           0.0
work_experience          0.0
requested_sum            0.0
main_agreement_amount    0.0
main_agreement_term      0.0
requested_period_days    0.0
requested_amount         0.0
req_app_amount           0.0
approved_amount          0.0
period_days              0.0
days_finish_loan         0.0
ag                       0.0
cnt_ext                  0.0
term                     0.0
price                    0.0
elecs_sum                0.0
recurents_sum            0.0
tamount                  0.0
issues                   0.0
principal                0.0
interest                 0.0
overdue_interest         0.0
overdue_fee              0.0
nbki_score               0.0
payment_frequency        0.0
status                   0.0
loan_id                  0.0
client_id                0.0
source                   0.0
first_source             0.0
interface                0.0
type                     0.0
repayment_type           0.0
client_type              0.0
settlement    

#### DataClean

In [126]:
class DataCleanerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, region_column=None, region_reference=None):
        self.missing_cols = []
        self.region_column = region_column
        self.region_reference = region_reference
        self.reference_list = []
        self.label_encoders = {}

    def fit(self, X, y=None):
        # Сохраняем список колонок с пропущенными значениями для добавления индикаторов
        self.missing_cols = X.columns[X.isnull().any()].tolist()

        if self.region_column and self.region_reference:
            reference = pd.read_csv(self.region_reference)
            reference["region_name"] = reference["region_name"].str.lower().str.strip()
            self.reference_list = reference["region_name"].tolist()

        return self

    def transform(self, X):
        data = X.copy()
        
        # Добавляем индикаторы пропущенных значений перед заполнением пропусков
        data = self.add_missing_indicators(data)

        # Применяем функции для заполнения пропущенных значений
        data = self.fill_monthly_income(data)
        data = self.fill_work_experience(data)
        data = self.fill_requested_sum(data)
        data = self.fill_requested_amount_and_req_app_amount(data)
        data = self.fill_requested_period_days(data)
        data = self.fill_term_and_price(data)
        data = self.fill_elecs_sum_and_recurents_sum(data)
        data = self.fill_tamount(data)
        data = self.fill_nbki_score(data)
        data = self.fill_cnt_ext(data)
        data = self.fill_start_dt(data)
        data = self.fill_contact_cases(data)

        if self.region_column:
            data[self.region_column] = data[self.region_column].str.lower().str.strip()
            data[self.region_column] = data[self.region_column].apply(self.find_closest_match)

        data = self.process_settlement(data)

        # Генерация новых признаков
        data = self.generate_features(data)

        # Обработка дат
        data = self.process_dates(data)

        # Преобразование категориальных переменных
        data = self.encode_categorical_features(data)

        # Удаление ненужных признаков
        data = self.delete_bad_features(data)

        return data

    def fill_monthly_income(self, data):
        data['monthly_income'] = data.groupby('status')['monthly_income'].transform(
            lambda x: x.fillna(x.median()) if not np.isnan(x.median()) else x.fillna(data['monthly_income'].median())
        )
        return data

    def fill_work_experience(self, data):
        data['work_experience'] = data.groupby('status')['work_experience'].transform(
            lambda x: x.fillna(x.median()) if not np.isnan(x.median()) else x.fillna(data['work_experience'].median())
        )
        return data

    def fill_requested_sum(self, data):
        data['requested_sum'] = data['requested_sum'].fillna(
            data['approved_amount'] * (data['requested_sum'].median() / data['approved_amount'].median())
        )
        return data

    def fill_requested_amount_and_req_app_amount(self, data):
        data['requested_amount'] = data['requested_amount'].fillna(data['requested_sum'])
        data['req_app_amount'] = data['requested_amount'] - data['approved_amount']
        return data

    def fill_requested_period_days(self, data):
        data['requested_period_days'] = data.groupby('type')['requested_period_days'].transform(
            lambda x: x.fillna(x.median())
        )
        return data

    def fill_term_and_price(self, data):
        data['term'] = data['term'].fillna(0)
        data['price'] = data['price'].fillna(0)
        return data

    def fill_elecs_sum_and_recurents_sum(self, data):
        data['elecs_sum'] = data.groupby('repayment_type')['elecs_sum'].transform(
            lambda x: x.fillna(x.median())
        )
        data['recurents_sum'] = data.groupby('repayment_type')['recurents_sum'].transform(
            lambda x: x.fillna(x.median())
        )
        return data

    def fill_tamount(self, data):
        data['tamount'] = data['tamount'].fillna(data['principal'] + data['interest'] + data['issues'])
        return data

    def fill_nbki_score(self, data):
        data['nbki_score'] = data.groupby('status')['nbki_score'].transform(
            lambda x: x.fillna(x.median())
        )
        return data

    def fill_cnt_ext(self, data):
        data['cnt_ext'] = data['cnt_ext'].fillna(0)
        return data

    def add_missing_indicators(self, data):
        for col in self.missing_cols:
            data[f'{col}_missing'] = data[col].isnull().astype(int)
        return data

    def fill_contact_cases(self, data):
        data['contact_cases'] = data.groupby('repayment_type')['contact_cases'].transform(
            lambda x: x.fillna(x.median())
        )
        data['contact_cases'] = data['contact_cases'].fillna(data['contact_cases'].median())
        return data

    def fill_start_dt(self, data):
        data['start_dt'] = pd.to_datetime(data['start_dt'], errors='coerce')
        data['start_dt'] = data.groupby('client_id')['start_dt'].transform(
            lambda x: x.fillna(x.min())
        )
        data['start_dt'] = data['start_dt'].fillna(data['created_at'])
        return data

    def find_closest_match(self, region):
        match = process.extractOne(region, self.reference_list, scorer=fuzz.ratio)
        return match[0] if match and match[1] >= 60 else region  # Минимальный порог схожести - 60%

    def generate_features(self, data):
        # Платежное поведение
        data['payment_to_income_ratio'] = data['tamount'] / data['monthly_income']
        data['interest_to_principal_ratio'] = data['interest'] / data['principal']
        data['overdue_ratio'] = (data['overdue_interest'] + data['overdue_fee']) / data['principal']

        # Временные характеристики
        data['early_repayment'] = data['period_days'] - data['days_finish_loan']

        # Кредитное поведение
        data['approval_ratio'] = data['approved_amount'] / data['requested_amount']
        data['previous_extensions'] = data['cnt_ext'].fillna(0)
        data['had_extensions'] = data['cnt_ext'].notna().astype(int)

        # Признаки рискованности клиента
        data['risk_score'] = (
            (data['nbki_score'] * -1) +  # Инвертируем скор, чтобы высокие значения означали высокий риск
            (data['overdue_ratio'] * 2) +
            (data['had_extensions'] * 1.5)
        )

        # Поведенческие признаки
        data['digital_engagement'] = ((data['interface'] == 2) |
                                        (data['source'].isin([11, 12]))).astype(int)
        data['prefers_longer_terms'] = (data['requested_period_days'] > 
                                          data['requested_period_days'].mean()).astype(int)

        # Группируем по client_id и создаем новые признаки
        client_features = data.groupby('client_id').agg({
            'loan_id': 'count',  # Общее количество займов
            'days_finish_loan': 'mean',  # Средний срок закрытия займа
            'approved_amount': 'mean',  # Средняя сумма займа
            'cnt_ext': 'sum',  # Общее количество продлений
            'elecs_sum': 'sum',  # Сумма штрафов
            'created_at': 'max',  # Дата последнего займа
            'monthly_income': 'std',  # Вариация дохода
            'contact_cases': 'sum'  # Количество обращений в поддержку
        }).reset_index()

        client_features.rename(columns={
            'loan_id': 'total_loans',
            'days_finish_loan': 'avg_days_finish_loan',
            'approved_amount': 'avg_approved_amount',
            'cnt_ext': 'total_extensions',
            'elecs_sum': 'total_fees',
            'created_at': 'last_loan_date',
            'monthly_income': 'income_variation',
            'contact_cases': 'total_contacts'
        }, inplace=True)
        data = data.merge(client_features, on='client_id', how='left')
        
        # Финансовая нагрузка
        data['total_debt_burden'] = (data['elecs_sum'] + data['recurents_sum']) / data['monthly_income']
        data['disposable_income_ratio'] = 1 - data['total_debt_burden']

        return data

    def process_dates(self, data):
        date_columns = ['created_at', 'closed_at', 'start_dt']
        for col in date_columns:
            if col in data.columns:
                data[col] = pd.to_datetime(data[col], errors='coerce')  # Преобразуем в datetime
                data[col] = data[col].fillna(data['start_dt'])  # Заполняем отсутствующие значения датой из 'start_dt'
                # Преобразуем datetime в формат timestamp (количество секунд с начала эпохи)
                data[col] = data[col].astype('int64') // 10**9
        return data

    def encode_categorical_features(self, data):
        categorical_features = ['status', 'payment_frequency', 'source', 'first_source', 
                                'interface', 'type', 'repayment_type', 'settlement', 'client_type.1', 'region', 'gender']

        for feature in categorical_features:
            data[feature] = data[feature].astype(str).fillna("missing")  # Используем строку, чтобы избежать ошибки
            le = LabelEncoder()
            data[feature] = le.fit_transform(data[feature])
            self.label_encoders[feature] = le

        return data

    def clean_city_name(self, name):
        # Извлечение последнего слова
        name = name.split()[-1].lower().strip()
        # Удаление нежелательных символов (всего, что не буквы и не цифры)
        name = re.sub(r'[^а-яa-z0-9]', '', name)
        
        return name

    def process_settlement(self, data):
        data['settlement'] = data['settlement'].apply(self.clean_city_name)
        threshold = 400
        data_region_counts = data['settlement'].value_counts()
        data['settlement'] = data['settlement'].apply(lambda x: x if data_region_counts[x] > threshold else 'другие')

        return data

    def delete_bad_features(self, data):
        columns = ['loan_id', 'client_id']
        data.drop(columns=columns, inplace=True)
        
        return data

In [135]:
# Пример использования
start_time = time.time()
transformer = DataCleanerTransformer(
    region_column="region", region_reference="/kaggle/input/region-reference/region_reference.csv"
)
# Файл region_reference содержит правильно написанные названия регионов, исправление работает, так что, находятся 
# ближайше возможные записи из region_reference и по ним заменяются записи из train и test

train = transformer.fit_transform(train)
test = transformer.fit_transform(test)

end_time = time.time()
elapsed_time = round(end_time - start_time, 2)
print(f"\nElapsed Time: {elapsed_time} seconds")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/region-reference/region_reference.csv'

Определим класс для предобработки данных, который выполняет:

- Очистку текстовых признаков:
— Для столбцов, таких как region и settlement, производится нормализация, удаление лишних слов (например, «область», «край» и т.д.) и спецсимволов, а также устранение дублирования слов.
— Для столбца settlement дополнительно создаётся новый признак settlement_category, определяемый с помощью регулярных выражений, классифицирующих тип населённого пункта (город, поселок городского типа, село).

- Обработку временных признаков:
— Из столбца created_at извлекаются базовые временные признаки, такие как месяц создания займа и день недели.
— При группировке по client_id вычисляются промежутки времени между займами, количество займов, а также производные признаки, такие как время с первого или до последнего займа и частота займов.

- Обработку финансовых признаков:
— Рассчитывается коэффициент, отношение одобренной суммы к запрашиваемой (approved_requested_ratio), что позволяет выявлять финансовые аномалии и соотношения.

- Обработку пропусков:
— Пропущенные значения в числовых признаках заполняются медианой (за исключением идентификаторов), что обеспечивает корректность дальнейшего анализа и моделирования.

- Кодирование категориальных признаков:
— Для признаков с небольшим числом уникальных значений применяется one-hot кодирование (с использованием OneHotEncoder), а для остальных — порядковое кодирование (OrdinalEncoder).
— Кроме того, создаётся комбинированный признак status_client_type, объединяющий информацию из нескольких исходных столбцов.

In [133]:
class DataPreprocessorOptimized:
    """
    Класс для предобработки данных.
    Выполняет:
      - Очистку текстовых признаков
      - Обработку временных и финансовых признаков
      - Обработка пропусков
      - Кодирование категориальных признаков
    """
    def __init__(self):
        # Инициализируем регулярные выражения и энкодеры
        self._init_regex_patterns()
        self._init_encoders()

    def _init_regex_patterns(self):
        """
        Инициализирует регулярные выражения для очистки текстовых признаков,
        таких как 'region' и 'settlement'. Здесь задаются слова, которые необходимо удалить,
        а также шаблоны для классификации населённых пунктов.
        """
        remove_words = [
            'обл', 'область', 'край', 'народная', 'респ', 'г',
            'республика', 'аобл', 'район', 'ао', 'автономный округ',
            'югра', 'якутия', 'кузбасс', 'алания', 'чувашия'
        ]
        self.pattern_remove_words = re.compile(r'\b(?:' + '|'.join(remove_words) + r')\b', re.I)
        self.pattern_non_word = re.compile(r'[^\w\s]', re.I)
        self.pattern_sakhalin = re.compile(r'\bсахалин\b', re.I)
        # Шаблоны для определения типа населённого пункта:
        # 2 – для поселков городского типа, 3 – для поселков/сел, 1 – для городов.
        self.settlement_patterns = {
            2: re.compile(r'\b(пгт|посёлок городского типа|поселок городского типа)\b', re.I),
            3: re.compile(r'\b(п|рп|поселок|посёлок|с|сп|село)\b', re.I),
            1: re.compile(r'\b(г|город)\b', re.I)
        }

    def _init_encoders(self):
        """
        Инициализирует энкодеры для категориальных признаков:
          - onehot_encoder для столбцов с небольшим числом уникальных значений.
          - ordinal_encoder для остальных.
        """
        self.onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
        self.ordinal_encoder = OrdinalEncoder()

    def _process_text_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Обрабатывает текстовые признаки:
          - Нормализует столбец 'region': приводит к нижнему регистру, удаляет лишние слова, спецсимволы,
            убирает дублирование слов.
          - Обрабатывает 'settlement': нормализует, создает категорию населённого пункта, а также
            вычисляет региональную популярность и индикатор для Москвы/СПБ.
        """
        # Обработка региона
        if 'region' in df.columns:
            df['region'] = (
                df['region']
                .str.lower()
                .str.replace(self.pattern_remove_words, '', regex=True)
                .str.replace(self.pattern_non_word, '', regex=True)
                .str.replace(r'\s+', ' ', regex=True)
                .str.replace(self.pattern_sakhalin, 'сахалинская', regex=True)
                .apply(lambda x: ' '.join(dict.fromkeys(x.split())))
                .str.strip()
            )

        # Обработка признака settlement
        if 'settlement' in df.columns:
            s = df['settlement'].str.lower()
            conditions = [
                s.str.contains(self.settlement_patterns[2], regex=True),
                s.str.contains(self.settlement_patterns[3], regex=True),
                s.str.contains(self.settlement_patterns[1], regex=True)
            ]
            df['settlement_category'] = np.select(conditions, [2, 3, 1], default=4)

        return df

    def _process_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Обрабатывает временные признаки:
          - Извлекает базовые признаки из 'created_at': created_month, created_dayofweek.
          - Использует групповые преобразования (transform) для вычисления:
              * time_between_loans: разница в днях между последовательными займами.
              * time_between_loans_mean: среднее значение time_between_loans по клиенту.
              * first_loan, last_loan, loan_count: минимальная и максимальная дата займа и количество займов.
              * time_since_first_loan и time_since_last_loan: производные признаки.
              * loan_frequency: частота займов.
        """
        # Сортировка по client_id и created_at
        df = df.sort_values(['client_id', 'created_at'])
        # Вычисляем разницу по времени между займами для каждой группы
        df['time_between_loans'] = df.groupby('client_id')['created_at'].diff().dt.days

        # Группируем по всему DataFrame по 'client_id'
        g = df.groupby('client_id')
        # Агрегируем нужные столбцы
        agg_df = g.agg({
            'time_between_loans': 'mean',
            'created_at': ['min', 'max'],
            'loan_id': 'size'
        }).reset_index()
        # Приводим многоуровневый индекс столбцов к плоскому виду
        agg_df.columns = ['client_id', 'time_between_loans_mean', 'first_loan', 'last_loan', 'loan_count']

        # Объединяем агрегированные данные с исходным DataFrame
        df = df.merge(agg_df, on='client_id', how='left')
        df['time_since_first_loan'] = (df['created_at'] - df['first_loan']).dt.days
        df['time_since_last_loan'] = (df['last_loan'] - df['created_at']).dt.days
        df['loan_frequency'] = df['loan_count'] / (((df['last_loan'] - df['first_loan']).dt.days / 365) + 1e-6)
        df.drop(columns=['first_loan', 'last_loan'], inplace=True)
        # Базовые временные признаки
        df['created_month'] = df['created_at'].dt.month
        df['created_dayofweek'] = df['created_at'].dt.dayofweek
        return df

    def _process_financial_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Обрабатывает финансовые признаки:
          - Вычисляет коэффициенты:
                approved_requested_ratio,
        """

        if 'approved_amount' in df.columns and 'requested_amount' in df.columns:
            df['approved_requested_ratio'] = df['approved_amount'] / (df['requested_amount'] + 1e-6)
        return df

    def _handle_missing_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Заполняет пропуски медианой для всех числовых признаков, кроме идентификаторов.
        """
        numeric_cols = df.select_dtypes(include=np.number).columns.difference(['client_id', 'loan_id'])
        if not numeric_cols.empty:
            medians = df[numeric_cols].median()
            df[numeric_cols] = df[numeric_cols].fillna(medians)

        return df

    def _encode_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Кодирует категориальные признаки:
          - Создает комбинированный признак 'status_client_type'.
          - Для признаков с ≤10 уникальными значениями применяется one-hot кодирование; новые столбцы добавляются,
            а исходные колонки остаются.
          - Для остальных создается новый столбец с порядковым кодированием (с суффиксом '_ordinal').
        """
        if 'status' in df.columns and 'client_type' in df.columns:
            df['status_client_type'] = df['status'].astype(str) + "_" + df['client_type'].astype(str)
            df['status_client_type'] = df['status_client_type'].astype('category')

        cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in cat_cols:
            # Если в столбце мало уникальных значений, применяем one-hot кодирование
            if df[col].nunique() <= 10:
                self.onehot_encoder.fit(df[[col]])
                encoded = self.onehot_encoder.transform(df[[col]])
                new_cols = self.onehot_encoder.get_feature_names_out([col])
                # Добавляем новые столбцы с закодированными признаками, исходный столбец остается
                df = pd.concat([
                    df,
                    pd.DataFrame(encoded, columns=new_cols, index=df.index)
                ], axis=1)
            else:
                # Для остальных создаем новый столбец с порядковым кодированием
                df[col + '_ordinal'] = self.ordinal_encoder.fit_transform(df[[col]]).astype(np.int32)

        return df

    def preprocess_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Выполняет последовательную предобработку данных.
        Шаги:
          1. Обработка текстовых признаков (region, settlement).
          2. Обработка временных признаков (создание базовых временных признаков и групповых статистик).
          3. Обработка финансовых признаков (расчет коэффициентов и производных признаков).
          4. Обработка пропусков (заполнение медианой).
          5. Кодирование категориальных признаков.

        После каждого шага вызывается сборщик мусора для оптимизации памяти.
        """
        processing_steps = [
            self._process_text_features,
            self._process_time_features,
            self._process_financial_features,
            self._handle_missing_data,
            self._encode_features,
        ]

        with tqdm(processing_steps, desc="Обработка данных") as pbar:
            for step in pbar:
                df = step(df).copy()
                pbar.set_postfix(shape=df.shape)
                gc.collect()

        return df

In [134]:
preprocessor = DataPreprocessorOptimized()
processed_train = preprocessor.preprocess_data(train.compute())

AttributeError: 'DataFrame' object has no attribute 'compute'

### Data Preparating

### TrainModel

### Result

### Submission